In [55]:
pip install pandas scikit-learn matplotlib tensorflow

In [56]:
import os
import pandas as pd
import shutil
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam


In [57]:
df = pd.read_csv('trainLabels.csv')

diagnosis_map = {
    0: 'No_DR',
    1: 'Mild',
    2: 'Moderate',
    3: 'Severe',
    4: 'Proliferative_DR'
}
df['label'] = df['level'].map(diagnosis_map)
df.sample(n=20000,random_state=42)
print(df.head())
train_df, val_df = train_test_split(df, test_size=0.40, stratify=df['level'], random_state=42)
train_df, test_df = train_test_split(train_df, test_size=0.15/0.85, stratify=train_df['level'], random_state=42)


      image  level  label
0   10_left      0  No_DR
1  10_right      0  No_DR
2   13_left      0  No_DR
3  13_right      0  No_DR
4   15_left      1   Mild


In [58]:
base_dir = 'retina_data'
src_dir = 'resized_train'
classes = list(diagnosis_map.values())

for split in ['train', 'val', 'test']:
    for cls in classes:
        os.makedirs(os.path.join(base_dir, split, cls), exist_ok=True)

def copy_images(dataframe, subset):
    for _, row in dataframe.iterrows():
        img_name = row['image'] + '.jpeg'
        label = row['label']
        src = os.path.join(src_dir, img_name)
        dst = os.path.join(base_dir, subset, label, img_name)
        if os.path.exists(src):
            shutil.copy(src, dst)

copy_images(train_df, 'train')
copy_images(val_df, 'val')
copy_images(test_df, 'test')

In [59]:
img_size = 224
batch_size = 32

train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    shear_range=0.1
)

val_test_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(
    os.path.join(base_dir, 'train'),
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

val_data = val_test_gen.flow_from_directory(
    os.path.join(base_dir, 'val'),
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

test_data = val_test_gen.flow_from_directory(
    os.path.join(base_dir, 'test'),
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)


Found 17355 images belonging to 5 classes.
Found 14051 images belonging to 5 classes.
Found 3720 images belonging to 5 classes.


In [60]:
base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

for layer in base_model.layers[:-50]:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(len(classes), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])


In [61]:
work = model.fit(train_data, validation_data=val_data, epochs=10)

loss, acc = model.evaluate(test_data)
print(f"✅ Test Accuracy: {acc * 100:.2f}% | Loss: {loss:.4f}")
model.save('dr_model.h5')

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
543/543 ━━━━━━━━━━━━━━━━━━━━ 383s 663ms/step - accuracy: 0.6110 - loss: 1.2247 - val_accuracy: 0.7296 - val_loss: 0.8482
Epoch 2/10
543/543 ━━━━━━━━━━━━━━━━━━━━ 346s 638ms/step - accuracy: 0.7075 - loss: 0.9212 - val_accuracy: 0.7410 - val_loss: 0.7846
Epoch 3/10
543/543 ━━━━━━━━━━━━━━━━━━━━ 338s 622ms/step - accuracy: 0.7347 - loss: 0.8279 - val_accuracy: 0.7482 - val_loss: 0.7735
Epoch 4/10
543/543 ━━━━━━━━━━━━━━━━━━━━ 340s 627ms/step - accuracy: 0.7424 - loss: 0.7969 - val_accuracy: 0.7506 - val_loss: 0.8147
Epoch 5/10
543/543 ━━━━━━━━━━━━━━━━━━━━ 336s 618ms/step - accuracy: 0.7470 - loss: 0.7824 - val_accuracy: 0.7555 - val_loss: 0.7496
Epoch 6/10
543/543 ━━━━━━━━━━━━━━━━━━━━ 338s 623ms/step - accuracy: 0.7505 - loss: 0.7596 - val_accuracy: 0.7540 - val_loss: 0.7741
Epoch 7/10
543/543 ━━━━━━━━━━━━━━━━━━━━ 340s 626ms/step - accuracy: 0.7500 - loss: 0.7603 - val_accuracy: 0.7568 - val_loss: 0.7432
Epoch 8/10
543/543 ━━━━━━━━━━━━━━━━━━━━ 340s 626ms/step - accuracy: 0.7566 -

✅ Test Accuracy: 75.40% | Loss: 0.7561
